In [1]:
from pathlib import Path
import duckdb


SYNTHEA = Path("../synthea/data/pregnancy")
SYNTHEA_PARQUET = Path("./output/synthea-parquet/")
SYNTHEA_DB = Path("./output/synthea.duckdb")

In [2]:
# ingest synthea data, storing intermedia parquets as well
with duckdb.connect((SYNTHEA_DB).as_posix()) as conn:
    for file_ in SYNTHEA.glob("*.ndjson"):
        duckdb.read_json(file_.as_posix()).to_parquet((SYNTHEA_PARQUET / f"{file_.stem}.parquet").as_posix())
    for file_ in SYNTHEA_PARQUET.glob("*.parquet"):
        stmt = f"""
            DROP TABLE IF EXISTS {file_.stem};
            CREATE TABLE {file_.stem} AS SELECT * FROM '{file_.as_posix()}'
            """
        conn.execute(stmt)

IOException: IO Error: Cannot open file "/workspaces/hdc-furahi/notebooks/output/synthea.duckdb": No such file or directory

In [2]:
# using Relational API as it incremental query building
# https://duckdb.org/docs/api/python/relational_api
# read-only for analytics
conn = duckdb.connect(database=SYNTHEA_DB.as_posix(), read_only=True)

## Unnesting

In [ ]:
# you can't call UNNEST recursively
# this will throw InvalidInputException
# conn.sql("""
#     SELECT UNNEST(UNNEST([[1,2,3],[4,5]]))
# """)

# you need to do this
conn.sql("""
    WITH temp AS (SELECT UNNEST([[1,2,3],[4,5]]) AS a)
    SELECT UNNEST(a) FROM temp
""")

In [ ]:
# or this
conn.sql("""
    SELECT UNNEST(t)
    FROM (
        SELECT UNNEST([[1,2,3],[4,5]]) as t
        )
""")

## Creating flat table of all encounters per patient
First step in reconstructing the patient journey, is to create a list of all encounters and procedures per patient in long format. Using duckdb, this involves multiple CTEs to UNNEST arrays and access STRUCTS. Below a first shot, it feels very clunky. Surely there must be a better way.

In [33]:
conn.sql("""
    PRAGMA table_info('Encounter')
""")

┌───────┬─────────────────┬───────────────────────────────────────────────────────────┬─────────┬────────────┬─────────┐
│  cid  │      name       │                           type                            │ notnull │ dflt_value │   pk    │
│ int32 │     varchar     │                          varchar                          │ boolean │  varchar   │ boolean │
├───────┼─────────────────┼───────────────────────────────────────────────────────────┼─────────┼────────────┼─────────┤
│     0 │ resourceType    │ VARCHAR                                                   │ false   │ NULL       │ false   │
│     1 │ id              │ UUID                                                      │ false   │ NULL       │ false   │
│     2 │ meta            │ STRUCT(profile VARCHAR[])                                 │ false   │ NULL       │ false   │
│     3 │ identifier      │ STRUCT("use" VARCHAR, "system" VARCHAR, "value" UUID)[]   │ false   │ NULL       │ false   │
│     4 │ status          │ VARC

In [42]:
conn.sql("""
    WITH temp1 AS (
        SELECT
            id,
            struct_extract(UNNEST(type), 'coding') AS coding
        FROM Encounter
    ),
    temp2 AS (
        SELECT
            id,
            UNNEST(temp1.coding) AS coding
        FROM temp1
    ),
    temp3 AS (
        SELECT
            id AS encounterId,
            coding.*
        FROM temp2
    )
    SELECT
        regexp_split_to_array(e.subject.reference, '/')[-1] AS patientId,
        p.birthDate,
        regexp_split_to_array(serviceProvider.reference, '\|')[-1] AS organizationId,
        temp3.*,
        CAST(e.period.start AS TIMESTAMP) AS period_start,
        CAST(e.period.end AS TIMESTAMP) AS period_end,

    FROM temp3
    LEFT JOIN Encounter e ON e.id = temp3.encounterId
    LEFT JOIN Patient p ON p.id = regexp_split_to_array(e.subject.reference, '/')[-1]

""")

┌──────────────────────┬────────────┬───┬──────────────────────┬─────────────────────┬─────────────────────┐
│      patientId       │ birthDate  │ … │       display        │    period_start     │     period_end      │
│       varchar        │    date    │   │       varchar        │      timestamp      │      timestamp      │
├──────────────────────┼────────────┼───┼──────────────────────┼─────────────────────┼─────────────────────┤
│ 093c96cc-54ef-9087…  │ 1969-11-21 │ … │ General examinatio…  │ 2013-12-06 17:01:35 │ 2013-12-06 17:16:35 │
│ cffc0afe-15f1-9dde…  │ 1969-11-09 │ … │ General examinatio…  │ 2013-11-24 22:13:38 │ 2013-11-24 22:28:38 │
│ c9c90c6b-d258-b628…  │ 2010-08-05 │ … │ Well child visit (…  │ 2013-07-11 08:10:17 │ 2013-07-11 08:25:17 │
│ 94f5dc9e-c391-a203…  │ 1987-09-17 │ … │ General examinatio…  │ 2015-11-26 07:53:42 │ 2015-11-26 08:08:42 │
│ bb589a13-5469-624f…  │ 1965-08-18 │ … │ General examinatio…  │ 2013-09-11 09:56:50 │ 2013-09-11 10:11:50 │
│ 05efa25e-2f63-c2a

In [4]:
conn.close()
